In [1]:
import os
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List, Tuple, NamedTuple, Callable
import scml
import mylib

In [2]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [3]:
%%time
df = pd.read_csv("input/js18/train.csv", engine="c", low_memory=False)
tmp = pd.read_csv("input/js18/test.csv", engine="c", low_memory=False)
df = df.append(tmp, ignore_index=True)
df.rename(columns={"comment_text": "text"}, inplace=True)
cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
df.drop(columns=["id"] + cols, inplace=True)
df.drop_duplicates(["text"], keep='first', inplace=True, ignore_index=True)
#df[cols] = df[cols].astype(np.int8) 
#df["label_sum"] = df["toxic"] + df["severe_toxic"] + df["obscene"] + df["threat"] + df["insult"] + df["identity_hate"]
#df.drop(index=df[df.label_sum == 0].index, inplace=True)
#df.drop(columns=["label_sum"], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312735 entries, 0 to 312734
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    312735 non-null  object
dtypes: object(1)
memory usage: 2.4+ MB
Wall time: 2.14 s


In [4]:
df["length"] = df["text"].str.len()
df["length"].describe(percentiles=percentiles)

count    312735.000000
mean        379.773262
std         591.767791
min           1.000000
1%           20.000000
5%           30.000000
10%          42.000000
20%          71.000000
30%         105.000000
40%         145.000000
50%         193.000000
60%         260.000000
70%         350.000000
80%         502.000000
90%         848.000000
95%        1314.000000
99%        3487.000000
max        5000.000000
Name: length, dtype: float64

In [5]:
df = df[(df["length"] > 30) & (df["length"] < 848)].copy()
df.drop(columns=["length"], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 265087 entries, 0 to 312734
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    265087 non-null  object
dtypes: object(1)
memory usage: 4.0+ MB


In [6]:
scml.find_missing_values(df)

,Total,Percent,Type
text,0,0.0,object


# Preprocess Text
Speed dropped from 2400 to 600 it/s

In [7]:
def pre1(row) -> str:
    return mylib.pre1(row["text"])


def pre2(row) -> str:
    return mylib.pre2(row["text1"])


def pre3(row) -> str:
    return mylib.pre3(row["text2"])


col = "text1"
print(col)
df[col] = df.progress_apply(pre1, axis=1)
col = "text2"
print(col)
df[col] = df.progress_apply(pre2, axis=1)
col = "text3"
print(col)
df[col] = df.progress_apply(pre3, axis=1)

text1


100%|██████████████████████████████████████| 265087/265087 [09:24<00:00, 469.36it/s]


text2


100%|█████████████████████████████████████| 265087/265087 [3:23:01<00:00, 21.76it/s]


text3


100%|██████████████████████████████████████| 265087/265087 [44:06<00:00, 100.16it/s]


In [8]:
# Drop texts that are too short
df = df[df["text1"].str.len() > 20].copy()

In [9]:
cols = ["text", "text1", "text2", "text3"]
df[cols].sample(20).head(20)

,text,text1,text2,text3
151999,"♥Valentine's Day Couples♥\n(alpahbetical order by last names)\nALTIERI, ANTHONY & ALLYSON FETZER\nBENEDETTO, DAVID & RONALD VARN\nCRIBBEN, EVAN & CHING SHIN SHOO\nDeLONG, ANDREW & JOEL TEAL\nFETZER, ALLISON & MITCH MUIR\nGILL, JEFFERY & MARY J. WANA\nHARTZELL, TIMOTHY & LAURA LEVITT\nICHIKAWA, SHIZUYO & NATHANIEL HUMPHREY\nKARNUTH, GREGORY & MICHEAL KOHLER\nLASLO, ERICA & DANIEL LEEK\nMADDAMMA, JAMES & DENNY LEEK\nPELLEGRINI, JORDAN & KATIE BARTON\nQIN, SIMON & SHIZUYO ICHIKAWA\nRARESHEID, KARA & BRIAN RASILE\nSCHAUB, MEGAN & JOHN RUTH\nSCHILSTRA, KEITH & DYLAN ROSSETT\nSICH, ANNA & HOMER RICHARDS\nTEAL, JOEL & NICHOLAS SINCERE\nUITTO, MATTHEW & VID. E. OJUEGOS","(heart suit) Valentine's Day Couples (heart suit) (alpahbetical order by last names) ALTIERI, ANTHONY & ALLYSON FETZER BENEDETTO, DAVID & RONALD VARN CRIBBEN, EVAN & CHING SHIN SHOO DeLONG, ANDREW & JOEL TEAL FETZER, ALLISON & MITCH MUIR GILL, JEFFERY & MARY J. WANA HARTZELL, TIMOTHY & LAURA LEVITT ICHIKAWA, SHIZUYO & NATHANIEL HUMPHREY KARNUTH, GREGORY & MICHEAL KOHLER LASLO, ERICA & DANIEL LEEK MADDAMMA, JAMES & DENNY LEEK PELLEGRINI, JORDAN & KATIE BARTON QIN, SIMON & SHIZUYO ICHIKAWA RARESHEID, KARA & BRIAN RASILE SCHAUB, MEGAN & JOHN RUTH SCHILSTRA, KEITH & DYLAN ROSSETT SICH, ANNA & HOMER RICHARDS TEAL, JOEL & NICHOLAS SINCERE UITTO, MATTHEW & VID. E. OJUEGOS","(heart suit) Valentine's Day Couples (heart suit) (alpahbetical order by last names) ALTIERI, ANTHONY & ALLYSON FETZER BENEDETTO, DAVID & RONALD VARN CRIBBEN, EVAN & CHING SHIN SHOO DeLONG, ANDREW & JOEL TEAL FETZER, ALLISON & MITCH MUIR GILL, JEFFERY & MARY J. WANA HARTZELL, TIMOTHY & LAURA LEVITT ICHIKAWA, SHIZUYO & NATHANIEL HUMPHREY KARNUTH, GREGORY & MICHEAL KOHLER LASLO, ERICA & DANIEL LEEK MADDAMMA, JAMES & DENNY LEEK PELLEGRINI, JORDAN & KATIE BARTON QIN, SIMON & SHIZUYO ICHIKAWA RARESHEID, KARA & BRIAN RASILE SCHAUB, MEGAN & JOHN RUTH SCHILSTRA, KEITH & DYLAN ROSSETT SICH, ANNA & HOMER RICHARDS TEAL, JOEL & NICHOLAS SINCERE UITTO, MATTHEW & VID. E. OJUEGOS",heart suit valentine s day couples heart suit alpahbetical order by last name altieri anthony allyson fetzer benedetto david ronald varn cribben evan ching shin shoo delong andrew joel teal fetzer allison mitch muir gill jeffery mary j wana hartzell timothy laura levitt ichikawa shizuyo nathaniel humphrey karnuth gregory micheal kohler laslo erica daniel leek maddamma james denny leek pellegrini jordan katie barton qin simon shizuyo ichikawa raresheid kara brian rasile schaub megan john ruth schilstra keith dylan rossett sich anna homer richards teal joel nicholas sincere uitto matthew vid e ojuegos
193499,"Huge confusion here. The Silvia sung by Bjorling, Tauber , McCormack et alii is a Schubert song with Shakespeare's words. Oley Speaks may well have written a song entitled Sylvia, but it was not that one.","Huge confusion here. The Silvia sung by Bjorling, Tauber , McCormack et alii is a Schubert song with Shakespeare's words. Oley Speaks may well have written a song entitled Sylvia, but it was not that one.","Huge confusion here. The Silvia sung by Bjorling, Tauber , McCormack et alii is a Schubert song with Shakespeare's words. Oley Speaks may well have written a song entitled Sylvia, but it was not that one.",huge confusion here the silvia sing by bjorle tauber mccormack et alii be a schubert song with shakespeare s word oley speak may well have write a song entitle sylvia but it be not that one
13600,"Reply -Had the redirects been left in place, the redirect bots would have fixed them. Now that they are gone, they must now be recreated.","Reply -Had the redirects been left in place, the redirect bots would have fixed them. Now that they are gone, they must now be recreated.","Reply -Had the redirects been left in place, the redirect bots would have fixed them. Now that they are gone, they must now be recreated.",reply have the redirect be leave in place the redirect bot would have fix they now that they b

In [10]:
df[cols].sample(20).head(20)

,text,text1,text2,text3
269166,""" \n\n == A brownie for you! == \n\n {| style=""""background-color: #fdffe7; border: 1px solid #fceb92;"""" \n |style=""""vertical-align: middle; padding: 5px;"""" | \n |style=""""vertical-align: middle; padding: 3px;"""" | Hot Brownies \n |}""",""" == A brownie for you! == {| style=""""background-color: #fdffe7; border: 1px solid #fceb92;"""" |style=""""vertical-align: middle; padding: 5px;"""" | |style=""""vertical-align: middle; padding: 3px;"""" | Hot Brownies |}""",""" == A brownie for you! == {| style=""""background-color: #fdffe7; border: 1px solid #fceb92;"""" |style=""""vertical-align: middle; padding: 5px;"""" | |style=""""vertical-align: middle; padding: 3px;"""" | Hot Brownies |}""",a brownie for you style background color fdffe7 border 1px solid fceb92 style vertical align middle padding 5px style vertical align middle padding 3px hot brownies
165962,"The male shane \n What are you planning on doing here? If your intention is simply to create a nonsense page, or defame someone, I'm just going to delete it.","The male shane What are you planning on doing here? If your intention is simply to create a nonsense page, or defame someone, I'm just going to delete it.","The male shane What are you planning on doing here? If your intention is simply to create a nonse page, or defame someone, I am just going to delete it.",the male shane what be you plan on do here if your intention be simply to create a nonse page or defame someone i be just go to delete it
220283,"2005 (UTC) \n :::::::::I was going to write almost exactly the same reply, but I chickened out. ) — 03:33, 17 August","2005 (UTC):I was going to write almost exactly the same reply, but I chickened out.) - 03:33, 17 August","2005 (UTC):I was going to write almost exactly the same reply, but I chickened out.) - 03:33, 17 August",2005 utc i be go to write almost exactly the same reply but i chickene out 03 33 17 august
129350,"""\n\nYou're ignoring this """"real life"""" fact we keep trying to tell you about. We know (or at least I know) Daffy Duck is NOT a real person, Plucky Duck is NOT a real person, they have NO fathers, mothers, sisters, brothers. So because Plucky Duck is NOT a real person, Plucky (just like every single other cartoon character that says he/she/it does) can't be signed to a contract. However when he implies he can, he implies it that he has signed this contract in the """"real life"""" we keep trying to tell you about. And it is in this """"real life"""" where Plucky is Daffy's REAL father, while the one in the baby Plucky cartoon is just an """"actor"""" (hired in the """"real world"""" where this cartoon character """"signed a contract"""") """"playing"""" his father. """,""" You're ignoring this """"real life"""" fact we keep trying to tell you about. We know (or at least I know) Daffy Duck is NOT a real person, Plucky Duck is NOT a real person, they have NO fathers, mothers, sisters, brothers. So because Plucky Duck is NOT a real person, Plucky (just like every single other cartoon character that says he/she/it does) can't be signed to a contract. However when he implies he can, he implies it that he has signed this contract in the """"real life"""" we keep trying to tell you about. And it is in this """"real life"""" where Plucky is Daffy's REAL father, while the one in the baby Plucky cartoon is just an """"actor"""" (hired in the """"real world"""" where this cartoon character """"signed a contract"""") """"playing"""" his father. """,""" you are ignoring this """"real life"""" fact we keep trying to tell you about. We know (or at least I know) Daffy Duck is NOT a real person, Plucky Duck is NOT a real person, they have NO fathers, mothers, sisters, brothers. So because Plucky Duck is NOT a real person, Plucky (just like every single other cartoon character that says he/she/it does) cannot be signed to a contract. However when he implies he can, he implies it that he has signed this contract in the """

In [11]:
df[cols].sample(20).head(20)

,text,text1,text2,text3
223055,I'm guessing this an album cover... what Mike do you want regoniction that you uploaded a fair use album seen on every other fucking website on the entire web?,I'm guessing this an album cover... what Mike do you want regoniction that you uploaded a fair use album seen on every other fucking website on the entire web?,I am guessing this an album cover... what Mike do you want regoniction that you uploaded a fair use album seen on every other fucking website on the entire web?,i be guess this an album cover what mike do you want regoniction that you upload a fair use album see on every other fucking website on the entire web
284708,because J.delanoy likes cock in his mouth}},because J.delanoy likes cock in his mouth}},because J.delanoy likes cock in his mouth}},because j delanoy like cock in his mouth
186815,"Discussion of Shit Cuisine \n There are many people who eat shit...such as my sister. In some countries, shit is another word for poop. \n E.g. Jane eats a lot of dog shit.","Discussion of Shit Cuisine There are many people who eat shit...such as my sister. In some countries, shit is another word for poop. E.g. Jane eats a lot of dog shit.","Discussion of Shit Cuisine There are many people who eat shit...such as my sister. In some countries, shit is another word for [poop; faeces]. E.g. Jane eats a lot of dog shit.",discussion of shit cuisine there be many people who eat shit such as my sister in some country shit be another word for poop faece e g jane eat a lot of dog shit
194671,Who keeps on removing the fact that he's a climate denier? Stupid removing it: he is one of the most prominent climate deniers out there. (),Who keeps on removing the fact that he's a climate denier? Stupid removing it: he is one of the most prominent climate deniers out there. (),Who keeps on removing the fact that he is a climate denier? Stupid removing it: he is one of the most prominent climate deniers out there. (),who keep on remove the fact that he be a climate denier stupid remove it he be one of the most prominent climate denier out there
281223,I am awaiting a response from FT2.,I am awaiting a response from FT2.,I am awaiting a response from FT2.,i be await a response from ft2
72892,I am telling you for a last time: Do not get my words from the context. And do not make silly yourself that the name is somewhere written as it is now. Thant name on the Serbian Wiki is written on the Serbian alphabet. Is it clear to you now. And also on the bulgarian Wikipedia is said that he is Macedonian. Is it clear to you now. And also on the bulgarian Wikipedia is said that he is Macedonian. P.S. Give me sources or proofs that he was writing on Bulgarian,I am telling you for a last time: Do not get my words from the context. And do not make silly yourself that the name is somewhere written as it is now. Thant name on the Serbian Wiki is written on the Serbian alphabet. Is it clear to you now. And also on the bulgarian Wikipedia is said that he is Macedonian. Is it clear to you now. And also on the bulgarian Wikipedia is said that he is Macedonian. P.S. Give me sources or proofs that he was writing on Bulgarian,I am telling you for a last time: Do not get my words from the context. And do not make silly yourself that the name is somewhere written as it is now. Thant name on the Serbian [Wiki; a website or database developed collaboratively by an online community] is written on the Serbian alphabet. Is it clear to you now. And also on the bulgarian Wikipedia is said that he is Macedonian. P.S. Give me sources or proofs that he was writing on Bulgarian,i be tell you for a last time do not get my word from the context and do not make silly yourself that the name be somewhere write as it be now thant name on the serbian wiki a website or database develop collaboratively by an online community be write on the serbian alphabet be it clear to you now and also on the bulgarian wikipedia be say that he be macedonian p s give i so

# Review data

In [12]:
col = "worker"
df[col] = np.random.randint(1, 50, size=len(df), dtype=np.int8)
cols = ["worker", "text", "text1", "text2", "text3"]
df[cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264106 entries, 0 to 312734
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   worker  264106 non-null  int8  
 1   text    264106 non-null  object
 2   text1   264106 non-null  object
 3   text2   264106 non-null  object
 4   text3   264106 non-null  object
dtypes: int8(1), object(4)
memory usage: 10.3+ MB


In [13]:
%%time
df[cols].to_parquet("output/pre_js18.parquet", index=False)

Wall time: 1.26 s
